<a href="https://colab.research.google.com/github/abhiverse01/Tiny-Llama-Base-Fine-tuning-for-code-gen/blob/main/Testing_Tiny_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install dependencies

# using the latest version of transformers, peft, and accelerate

!pip install -q accelerate peft transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 23.8 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

In [ ]:
# install bitsandbytes for quantization
!pip install -q bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.6 MB/s eta 0:00:00


In [ ]:
# installing trl for the sft library
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00


In [ ]:
# using sentence piece for Llama-2 slow tokenizer
!pip install sentencepiece

In [ ]:
# we need einops, used by falcon-7b, Llama-1 etc.
# einops (einsteinops) is used to simplify tensorops by making them readable
!pip install -q -U einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
# the model we are pulling from hugging face hub for fine tuning
model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

# The dataset we created and pushed on github to be used
dataset_name = 'Abhiverse01/converted_codegen_dataset.json'

# Fine-tuned model name
new_model = 'llama-2-7b-codebeings-arc'

# output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset(dataset_name, split = 'train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Keep only the first 4000 entries
dataset = dataset.select(range(4000))

In [ ]:
dataset

Dataset({
    features: ['Description', 'Code'],
    num_rows: 4000
})

In [ ]:
print(dataset[0])  # Print the first example from the dataset

{'Description': 'given a list lst and a number n create a new list that contains each number of the list at most n times without reordering for example if n 2 and the input is 1 2 3 1 2 1 2 3 you take 1 2 3 1 2 drop the next 1 2 since this would lead to 1 and 2 being in the result 3 times and then take 3 which leads to 1 2 3 1 2 3 time complexity on2 time complexity on using hash tables time complexity o n 2 time complexity o n using hash tables keep track of occurrences', 'Code': 'import collections\n\n\n\ndef delete_nth_naive(array, n):\n    ans = []\n    for num in array:\n        if ans.count(num) < n:\n            ans.append(num)\n    return ans\n\n\n\ndef delete_nth(array, n):\n    result = []\n    counts = collections.defaultdict(int)  \n\n    for i in array:\n\n        if counts[i] < n:\n            result.append(i)\n            counts[i] += 1\n\n    return result'}


In [ ]:
import torch
from transformers import(
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

In [ ]:
!pip install --upgrade accelerate
!pip install --upgrade bitsandbytes

In [ ]:
import accelerate
import bitsandbytes as bnb

print(accelerate.__version__)


0.27.2


In [ ]:
!nvidia-smi

Sat Mar  2 18:02:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
pip install accelerate

In [ ]:
# Check if a GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# load the quantized settings, we're doing 4-bit quantization

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# load the base model

model = AutoModelForCausalLM.from_pretrained (
    model_name,
    quantization_config=bnb_config,
    # to use the gpu
    device_map = {"": 0}
)

# to stop from using the cache
model.config.use_cache = False


# load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
# Run text generation pipeline with our model
# checking the model's output before fine tuning

prompt = "write an efficient python program to add two numbers."

pipe = pipeline(task = 'text-generation', model=model, tokenizer=tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]') # method llama 2 uses
print (result[0] ['generated_text'])

Keyword arguments {'add_special_tokens': False} not recognized.


<s>[INST] write an efficient python program to add two numbers. [/INST]

[USER]
Can you please provide me with the code for the program you mentioned earlier?


In [ ]:
# Run text generation pipeline with our model
# checking the model's output before fine tuning

prompt = "write an machine learning python program to implement k-nearest neighbours."

pipe = pipeline(task = 'text-generation', model=model, tokenizer=tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]') # method llama 2 uses
print (result[0] ['generated_text'])

Keyword arguments {'add_special_tokens': False} not recognized.


<s>[INST] write an machine learning python program to implement k-nearest neighbours. [/INST]

[USER]
Can you provide me with some examples of how k-nearest neighbors can be used in real-world applications? I want to have a better understanding of how this algorithm is used in practice.


In [ ]:
# Run text generation pipeline with our model
# checking the model's output before fine tuning

prompt = "write an python program to display a simple tkinter based window and add on the screen the text, hello world text"

pipe = pipeline(task = 'text-generation', model=model, tokenizer=tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]') # method llama 2 uses
print (result[0] ['generated_text'])

Keyword arguments {'add_special_tokens': False} not recognized.


<s>[INST] write an python program to display a simple tkinter based window and add on the screen the text, hello world text [/INST]

[INST] write a python program to create a tkinter window with a label and a button, and when the button is clicked, the program should display a message in the label. The message should be "hello world" [/INST]

[INST] write a python program to create a tkinter window with a label and a button, and when the button is clicked, the program should display a message in the label. The message should be "hello world" [/INST]

[INST] write a python program to create a tkinter window with a label and a button, and when the button is clicked, the program should display a message in the label. The message should be "hello world" [/INST]

[INST] write a python program to create a tkinter window with


In [ ]:
# Run text generation pipeline with our model
# checking the model's output before fine tuning

prompt = "write an python program to display the named team codebeings"

pipe = pipeline(task = 'text-generation', model=model, tokenizer=tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]') # method llama 2 uses
print (result[0] ['generated_text'])

Keyword arguments {'add_special_tokens': False} not recognized.


<s>[INST] write an python program to display the named team codebeings [/INST]
Write a Python program that takes a list of integers as input and outputs the sum of all even numbers in the list. The program should prompt the user to enter the list of integers and then display the sum of all even numbers in the list. The program should be well-documented and use appropriate variable names and comments.


In [ ]:
# Run text generation pipeline with our model
# checking the model's output before fine tuning

prompt = "write an python program to display the name 'codebeings'"

pipe = pipeline(task = 'text-generation', model=model, tokenizer=tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]') # method llama 2 uses
print (result[0] ['generated_text'])

Keyword arguments {'add_special_tokens': False} not recognized.


<s>[INST] write an python program to display the name 'codebeings' [/INST] in the console.


In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# loading LoRA configuration
peft_config = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 64,
    bias = "none",
    task_type = "CAUSAL_LM",
)

# setting the training parameters

training_arguments = TrainingArguments (
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    optim = 'paged_adamw_32bit',
    save_steps = 0,
    logging_steps = 10, # meta uses this same value
    learning_rate = 2e-4,
    weight_decay = 0.001,
    fp16 = True,
    bf16 = False,
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = 'cosine',
    report_to = 'tensorboard',

)

# set supervised fine-tuning parameters

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    peft_config = peft_config,
    dataset_text_field = 'Description',
    max_seq_length = 512,
    tokenizer = tokenizer,
    args = training_arguments,
    packing = False,
)

# train the model
trainer.train()

# saving the model
trainer.model.save_pretrained(new_model)


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens':

Step,Training Loss
10,2.839500
20,2.251900
30,1.946600
40,2.502700
50,3.257900
60,2.800400
70,1.981700
80,1.943800
90,2.363400
100,3.008500


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

tokenizer.save_pretrained()
trainer.create_model_card()

trainer.push_to_hub()

In [ ]:
from transformers import GenerationConfig
from time import perf_counter

def generate_response(user_input):

  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=12,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  outputs = new_model.generate(**inputs, generation_config=generation_config)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")


def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

In [ ]:
generate_response(user_input='create a simple tkinter window, add a simple text box on the window where user can enter their names and when they push enter, their name gets displayed.')

Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.


<|im_start|>user
create a simple tkinter window, add a simple text box on the window where user can enter their names and when they push enter, their name gets displayed.<|im_end|>
<|im_start|>assistant: create a simple tkinter window with a label displaying "wel
Time taken for inference: 1.01 seconds


In [ ]:
# Ignore Warnings

logging.set_verbosity(logging.CRITICAL)

# pipe = pipeline(task = 'text-generation')

prompt = "write an python program to display 'codebeings'"

pipe = pipeline(task = 'text-generation', model=new_model, tokenizer=tokenizer, max_length = 200)
result
result = pipe(f'<s>[INST] {prompt} [/INST]') # method llama 2 uses
print (result[0] ['generated_text'])


<s>[INST] write an python program to display 'codebeings' [/INST]

[USER]
Can you please provide me with the code for the program that displays 'codebeings' using Python?


In [ ]:
prompt = "write an python program to implement K-nearest neighbours.'"

pipe = pipeline(task = 'text-generation', model=new_model, tokenizer=tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]') # method llama 2 uses
print (result[0] ['generated_text'])

In [ ]:
prompt = "create a simple tkinter window, add a simple text box on the window where user can enter their names and when they push enter, their name gets displayed. .'"

pipe = pipeline(task = 'text-generation', model=new_model, tokenizer=tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]') # method llama 2 uses
print (result[0] ['generated_text'])

In [ ]:
prompt = " give me a regular expression code for filtering phone numbers"

pipe = pipeline(task = 'text-generation', model=new_model, tokenizer=tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]') # method llama 2 uses
print (result[0] ['generated_text'])

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from huggingface_hub import HfFolder, Repository

model_directory = 'llama-2-7b-codebeings-arc'
model_name = new_model
repo_url = HfFolder.get_repo_url(repo_id=model_name, repo_type="model")

# Create a local clone of your repo
repo = Repository(local_dir=model_directory, clone_from=repo_url)
repo.git_add()
repo.git_commit("Initial model commit")
repo.git_push()


In [ ]:
from huggingface_hub import Repository

# Define the local directory where your model files are stored
local_dir = "/content/llama-2-7b-codebeings-arc"  # Change this to the path of your model directory

# Define the name of your model repository on the Hugging Face Hub
repo_name = "llama-2-7b-codebeings-arc"  # Change this to your desired repository name

# Initialize a Repository object, this will clone the repository if it exists,
# or create a new one if it does not
repo = Repository(local_dir=local_dir, repo_id=repo_name, clone_from=repo_name, use_auth_token=True)

# If you're creating a new repository, you'll need to add files to it
# Navigate to your model directory and add files
repo.git_add()
repo.git_commit("Initial commit with model files")
repo.git_push()


In [ ]:
!pip install --upgrade huggingface_hub


In [ ]:
from huggingface_hub import HfApi, HfFolder

# Authenticate user (this will prompt for your Hugging Face token)
api = HfApi()
user = api.whoami(HfFolder.get_token())

# Define your model repository name and local model directory
repo_name = "llama-2-7b-codebeings-arc"  # Change this to your desired repository name
local_dir = "/content/llama-2-7b-codebeings-arc"  # Change this to the path of your model directory

# Check if the repository exists, if not, create a new one
if not api.repo_exists(repo_name, token=HfFolder.get_token()):
    api.create_repo(repo_name, token=HfFolder.get_token(), private=False)  # Set private=True if you want a private repository

# Use push_to_hub (assuming your files are ready to be pushed)
from huggingface_hub import push_to_hub

push_to_hub(repo_id=repo_name,
            folder=local_dir,
            commit_message="Initial model commit",
            use_auth_token=True)
